In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math
import ipywidgets as widgets

In [2]:
x = sp.symbols('x')


def associated_legendre_polynomial(p, q, x):
    derivative = sp.diff(sp.exp(-x) * x ** (p + q), x, q)
    L = x ** -p * sp.exp(x) / math.factorial(q) * derivative
    return sp.lambdify(x, L.simplify(), 'numpy')

In [3]:
def symbolic_r_function(n, l):
    r = sp.symbols('r')
    a = 1
    p, q = 2 * l + 1, n - l - 1
    derivative_r = sp.diff(sp.exp(-x) * x ** (p + q), x, q)
    L = x ** -p * sp.exp(x) / math.factorial(q) * derivative_r

    R = sp.sqrt((2 / (n * a)) ** 3 * math.factorial(n - l - 1) / (2 * n * math.factorial(n + l))) * sp.exp(
        -r / (n * a)) * (2 * r / (n * a)) ** l * L.subs(x, 2 * r / (n * 1))

    return sp.lambdify(r, R.simplify(), 'numpy')


def symbolic_theta_function(l, m):
    x = sp.symbols('x')
    theta = sp.symbols('θ')
    P = (-1) ** m * (1 - x ** 2) ** (m / 2) * sp.diff(
        1 / (2 ** l * math.factorial(l)) * sp.diff((x ** 2 - 1) ** l, x, l), x, m)
    return sp.lambdify(theta, P.subs(x, sp.cos(theta)).simplify(), 'numpy')


def symbolic_phi_function(l, m):
    phi = sp.symbols('φ')
    A = ((2 * l + 1) / (4 * sp.pi) * math.factorial(l - m) / math.factorial(l + m)) ** 0.5 * sp.exp(sp.I * m * x)
    return sp.lambdify(phi, A.subs(x, phi).simplify(), 'numpy')

In [4]:
def hydrogen_2d_slice(n, l, m, slice2d=40, zoom=40, fig_size=10):
    grid_resolution = 100
    x = y = z = np.linspace(-zoom, zoom, grid_resolution)
    x, y, z = np.meshgrid(x, y, z)

    R = abs(symbolic_r_function(n, l)(np.sqrt(x ** 2 + y ** 2 + z ** 2)))
    Theta = abs(symbolic_theta_function(l, m)(np.arccos(z / np.sqrt(x ** 2 + y ** 2 + z ** 2)))) * abs(
        symbolic_phi_function(l, m)(np.arctan2(y, x)))

    wavefunction = R * Theta
    wavefunction[wavefunction < 0.0001] = 0
    
    fig = plt.figure()
    fig.set_figwidth(fig_size)
    fig.set_figheight(fig_size)
    
    ax = fig.add_subplot(111)
    ax.imshow(wavefunction[:, slice2d, :])
    plt.show()



n_input = widgets.IntText(value=3, description='n:')
l_input = widgets.IntText(value=2, description='l:')
m_input = widgets.IntText(value=0, description='m:')
zoom_input = widgets.IntText(value=25, description='Zoom:')
slice_input = widgets.IntText(value=50, description='3D Slice:')
fig_size_input = widgets.IntText(value=10, description='Figure size:')

widgets.interact(hydrogen_2d_slice, n=n_input, l=l_input, m=m_input,
                 zoom=zoom_input, slice2d=slice_input, fig_size=fig_size_input)

interactive(children=(IntText(value=3, description='n:'), IntText(value=2, description='l:'), IntText(value=0,…

<function __main__.hydrogen_2d_slice(n, l, m, slice2d=40, zoom=40, fig_size=10)>